In [1]:
import os

In [2]:
from extractDataSrc.Mdi import Mdi

In [3]:
# check if destination folders for images are present
destination_folders = ["data_processed/eit", "data_processed/mdi/mag", "data_processed/mdi/con" ,"data_processed/lasco/c2", "data_processed/lasco/c3", "data_processed/in_situ"]

for folder in destination_folders:
    os.makedirs(folder, exist_ok=True)

In [4]:
start_datetime = "2003-03-21 00:00:00"
stop_datetime = "2003-04-10 23:59:59"

### Extracting magnetograms and continuum MDI

In [7]:
mdi = Mdi(start_datetime, stop_datetime)
# mdi.extract_data("mag")
# works! :)